In [10]:
import h3
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from concurrent.futures import ProcessPoolExecutor

In [11]:
class H3ShapefileGenerator:
    def __init__(self, shapefile_path, output_path, resolution=9):
        self.shapefile_path = shapefile_path
        self.output_path = output_path
        self.resolution = resolution
        self.polygon = self.load_shapefile().geometry[0]

    def load_shapefile(self):
        return gpd.read_file(self.shapefile_path).to_crs("EPSG:4326")

    def convert_polygon_to_h3_indices(self):
        polygon_geojson = mapping(self.polygon)
        return h3.polyfill(polygon_geojson, self.resolution)

    @staticmethod
    def convert_h3_index_to_polygon(h3_index):
        try:
            coords = h3.h3_to_geo_boundary(h3_index)
            return Polygon(coord[::-1] for coord in coords)
        except Exception as e:
            print(f"Error converting H3 index to polygon: {e}")
            return None

    def generate_shapefile_from_h3_indices(self):
        hex_df = pd.DataFrame(self.convert_polygon_to_h3_indices(), columns=['hex9'])
        with ProcessPoolExecutor() as executor:
            hex_df['geometry'] = list(executor.map(self.convert_h3_index_to_polygon, hex_df['hex9']))
        hex_df.dropna(subset=['geometry'], inplace=True)
        gdf = gpd.GeoDataFrame(hex_df, geometry='geometry', crs=4326)
        print(gdf)
        gdf.to_file(self.output_path)
        print(f"Shapefile saved at {self.output_path}")


In [12]:
if __name__ == "__main__":
    generator = H3ShapefileGenerator(
        shapefile_path="../../shapefiles/zh_poly/zh_poly.shp",
        output_path='../../hex/h3_list.shp'
    )
    generator.generate_shapefile_from_h3_indices()

                  hex9                                           geometry
0      897ab475d03ffff  POLYGON ((51.65574 4.31006, 51.65567 4.31173, ...
1      897ab44f047ffff  POLYGON ((52.06742 4.34963, 52.06735 4.35129, ...
2      897ab68406fffff  POLYGON ((52.21172 4.48075, 52.21165 4.48241, ...
3      897ab40720bffff  POLYGON ((51.74769 4.02597, 51.74762 4.02764, ...
4      897ab440cafffff  POLYGON ((52.04300 4.24012, 52.04293 4.24178, ...
...                ...                                                ...
62340  897ab409833ffff  POLYGON ((51.87087 4.18754, 51.87080 4.18920, ...
62341  897ab6a9d0bffff  POLYGON ((52.04772 4.76415, 52.04765 4.76580, ...
62342  897ab40eec7ffff  POLYGON ((51.73714 4.10910, 51.73707 4.11076, ...
62343  897ab6a706fffff  POLYGON ((51.91391 4.61040, 51.91384 4.61205, ...
62344  897ab6a4a8bffff  POLYGON ((51.82972 4.65481, 51.82965 4.65647, ...

[62345 rows x 2 columns]
Shapefile saved at ../../hex/h3_list.shp
